In [ ]:
import concurrent
import numpy as np
import numba as nb
import dask.array as da
import time
import matplotlib.pyplot as plt
from math import isnan, inf

In [ ]:
def np_min_max(arr):
    return np.nanmin(arr), np.nanmax(arr)

def test_tpe(arr):
    thread_results = list()
    with concurrent.futures.ThreadPoolExecutor() as executor: 
         for result in executor.map(np_min_max, arr):
                thread_results.append(result)
    return np_min_max(thread_results)

def dask_min_max(arr):
    np.nanmin(arr).compute().item(), np.nanmax(arr).compute().item()

In [ ]:
@nb.njit(fastmath=True)
def _minmax_nan(x):
    maximum = -inf
    minimum = inf
    for i in x:
        if not isnan(i):
            if i > maximum:
                maximum = i
            if i < minimum:
                minimum = i
    return minimum, maximum

@nb.njit(parallel=True)
def _minmax_chunks_nan(x, chunk_ranges):
    overall_maxima = []
    overall_minima = []
    for i in nb.prange(chunk_ranges.shape[0]):
        start = chunk_ranges[i, 0]
        end = chunk_ranges[i, 1]
        chunk_minimum, chunk_maximum = _minmax_nan(x[start : end])
        overall_maxima.append(chunk_maximum)
        overall_minima.append(chunk_minimum)
    return min(overall_minima), max(overall_maxima)

def even_chunk_sizes(dividend, divisor):
    quotient, remainder = divmod(dividend, divisor)
    cells = [quotient for _ in range(divisor)]
    for i in range(remainder):
        cells[i] += 1
    return cells

def even_chunk_ranges(dividend, divisor):
    sizes = even_chunk_sizes(dividend, divisor)
    ranges = []
    start = 0
    for s in sizes:
        end = start + s
        ranges.append((start, end))
        start = end
    return ranges

def nanminmax_parallel(x, n_chunks):
    chunk_ranges = np.array([
        [start, end]
        for start, end
        in even_chunk_ranges(len(x), n_chunks)
    ], dtype=np.int64)
    return _minmax_chunks_nan(x, chunk_ranges)

In [ ]:
#chunked_flat_np_arr = np.split(flat, 16)
#%timeit test_tpe(chunked_flat_np_arr)

In [ ]:
#flat = np.random.rand(100000000)  # 100 million

In [ ]:
#%timeit np_min_max(flat)

In [ ]:
#%timeit minmax(flat)

In [ ]:
#%timeit nb_min_max(flat)

In [ ]:
#dask_arr = da.from_array(flat)
#%timeit dask_min_max(dask_arr)

In [ ]:
def time_multithreaded(arr, reps):
    start_setup = time.perf_counter()
    chunked_flat_np_arr = np.array_split(arr, 16)
    setup_time = time.perf_counter() - start_setup
    setup_time
    
    timings = list()
    for _ in range(reps):   
        t_0 = time.perf_counter()        
        test_tpe(chunked_flat_np_arr)
        t_e = time.perf_counter() - t_0
        timings.append(t_e*1000)    
    return {"time_avg": np.mean(timings), "timings": timings, "setup_time": setup_time*1000}  


def time_plain_np(arr, reps):
    setup_time = 0
    
    timings = list()
    for _ in range(reps):   
        t_0 = time.perf_counter()        
        np_min_max(arr)
        t_e = time.perf_counter() - t_0 
        timings.append(t_e*1000)    
    return {"time_avg": np.mean(timings), "timings": timings, "setup_time": setup_time}  


def time_dask(arr, reps):
    start_setup = time.perf_counter()
    dask_arr = da.from_array(arr)
    setup_time = time.perf_counter() - start_setup
    setup_time
    
    timings = list()
    for _ in range(reps):   
        t_0 = time.perf_counter()        
        dask_min_max(dask_arr)
        t_e = time.perf_counter() - t_0
        timings.append(t_e*1000)    
    return {"time_avg": np.mean(timings), "timings": timings, "setup_time": setup_time}  

def time_jit_parallelized(arr, reps):
    setup_time = 0
    
    timings = list()
    for _ in range(reps):   
        t_0 = time.perf_counter()        
        nanminmax_parallel(arr, 4)
        t_e = time.perf_counter() - t_0
        timings.append(t_e*1000)    
    return {"time_avg": np.mean(timings), "timings": timings, "setup_time": setup_time*1000}  

In [ ]:
def time_all(reps=2):
    timings = dict(reps=reps)
    for size in ARR_SIZES:
        timings[size] = dict()
        size_timing = timings[size]
        arr = np.random.rand(size).ravel()

        size_timing["plain"] = time_plain_np(arr, reps)
        size_timing["multithreaded"] = time_multithreaded(arr, reps)
        size_timing["dask"] = time_dask(arr, reps)
        size_timing["jit_parallel"] = time_jit_parallelized(arr, reps)
        time.sleep(0.1)
    return timings

def get_timings_for(timing_dict, method: str):
    timings = list()
    for size in ARR_SIZES:
        t = timing_dict[size][method]["time_avg"]
        timings.append(t)
    return timings

In [ ]:
ARR_SIZES = [1000, 10000, 100000, 1000000, 5000000, 25000000, 100000000, 250000000, 500000000, 1000000000]

In [ ]:
REPS = 1

In [ ]:
for size in ARR_SIZES:
    arr = np.random.rand(10)    
    nanminmax_parallel(arr, 2)


In [ ]:
timing_results = time_all(REPS)

In [ ]:
fig = plt.figure()
x = ARR_SIZES

y_plain = get_timings_for(timing_results, "plain")
plt.plot(x, y_plain, '--', color='blue', label='plain numpy')

y_dask = get_timings_for(timing_results, "dask")
plt.plot(x, y_dask, '--', color='orange', label='dask')

y_multithreaded = get_timings_for(timing_results, "multithreaded")
plt.plot(x, y_multithreaded, '--', color='purple', label='multithreaded')

y_jit = get_timings_for(timing_results, "jit_parallel")
plt.plot(x, y_jit, '--', color='black', label='jit_parallel')

plt.grid(axis='x', color='0.95')
plt.xscale('log')
plt.xlabel('Array size', fontsize=18)
plt.ylabel('Time taken (ms)', fontsize=16)
plt.legend(title='Calculation method:')
plt.title(f'AVG of {REPS} repetitions using an i7-3770 @ 4.3GHz', fontsize=10)
plt.suptitle('Benchmarking nanmin&nanmax calculation', fontsize=15)
#plt.set_title('axes title')
plt.show()

In [ ]:
fig = plt.figure()
x = ARR_SIZES

y_plain = get_timings_for(timing_results, "plain")
plt.plot(x, y_plain, '--', color='blue', label='plain numpy')

y_dask = get_timings_for(timing_results, "dask")
plt.plot(x, y_dask, '--', color='orange', label='dask')

y_multithreaded = get_timings_for(timing_results, "multithreaded")
plt.plot(x, y_multithreaded, '--', color='purple', label='multithreaded')

y_jit = get_timings_for(timing_results, "jit_parallel")
plt.plot(x, y_jit, '--', color='black', label='jit_parallel')

plt.grid(axis='x', color='0.95')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Array size', fontsize=18)
plt.ylabel('Time taken (ms)', fontsize=16)
plt.legend(title='Calculation method:')
plt.title(f'AVG of {REPS} repetitions using an i7-3770 @ 4.3GHz', fontsize=10)
plt.suptitle('Benchmarking nanmin&nanmax calculation', fontsize=15)
#plt.set_title('axes title')
plt.show()

In [ ]:
#arr = np.random.rand(1000000000)

In [ ]:
#%timeit minmax_chunks_nan(arr, 4)  # (c) Saltrock

In [ ]:
#%timeit np_min_max(arr)  #  np.min+np.max

In [ ]:
#%timeit nb_min_max(arr) #  deleting nans then np.min+np.max with JIT

In [ ]:
#chunked_flat_np_arr = np.split(arr, 16) #  Multithreading
#%timeit test_tpe(chunked_flat_np_arr)

In [ ]:
#dask_arr = da.from_array(arr)  # Multiprocessing
#%timeit dask_min_max(dask_arr)